In [26]:
import os
import pandas as pd
from dotenv import load_dotenv
import logging
import sshtunnel
from sshtunnel import SSHTunnelForwarder
import pymysql
import paramiko
from os.path import expanduser
load_dotenv()

True

In [27]:

CREDENTIALS = {
                 'SSH_HOST': os.getenv('SSH_HOST'),
                 'SSH_USER':os.getenv('SSH_USER'),
                 'SSH_PASSWORD':os.getenv('SSH_PASSWORD'), 
                 'DB_NAME' : os.getenv('DB_NAME'), 
                 'DB_USER' : os.getenv('DB_USER'),
                 'DB_PASSWORD': os.getenv('DB_PASSWORD'),
                 'LOCAL_HOST': os.getenv('LOCAL_HOST'),
                 'PKEY_DIR': os.getenv('PKEY_DIR'),
                 'HOME_PASSWORD': os.getenv('HOME_PASSWORD'),}

In [28]:
class DBOperations:

    def __init__(self):

        self.connection = None
        self.tunnel = None

    def open_ssh_tunnel(self, credentials, verbose=False):
        """Open an SSH tunnel and connect using a username and password.
        
        :param verbose: Set to True to show logging
        :return tunnel: Global SSH tunnel connection
        """

        home = expanduser('~')
        filepath = home+'/.ssh/id_rsa'
        password = '@Nsom4zrts'
        c = paramiko.SSHClient()
        c.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        if verbose:
            sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG
        self.tunnel = SSHTunnelForwarder(
            ('172.104.207.179', 22),
            ssh_username = 'zephania',
            ssh_pkey=paramiko.RSAKey.from_private_key_file(filename=filepath, password=password),
            ssh_password = 'SiJqLi4A9rWkcb2',
            remote_bind_address = ('127.0.0.1', 3306)
        )
        
        self.tunnel.start()

    def mysql_connect(self,credentials):
        """Connect to a MySQL server using the SSH tunnel connection
        
        :return connection: Global MySQL database connection
        """
    
        
        self.connection = pymysql.connect(
            host='127.0.0.1',
            user='zephania_zephania_dw',
            passwd='@zephania@dw',
            db='zephania_dwrecdb',
            port=self.tunnel.local_bind_port
        )


    def run_query(self, table_name):
        """Runs a given SQL query via the database connection.
        
        :param sql: MySQL query
        :return: Pandas dataframe containing results
        """
        if self.connection:
            sql = """SELECT * FROM {0} """.format(table_name)

            df = pd.read_sql_query(sql, self.connection)
            
            df.to_csv('../../../data/'+table_name+'.csv')
        
        else:
            raise ValueError("No database connection")
            

    def mysql_disconnect(self):
        """Closes the MySQL database connection.
        """
        if self.connection:  
            self.connection.close()
        else:
            raise ValueError("No connection to close")




In [29]:
dbo_obj = DBOperations()

In [30]:
dbo_obj.open_ssh_tunnel(credentials=CREDENTIALS)
dbo_obj.mysql_connect(credentials=CREDENTIALS)
table_names= ['drugs','customers','orders', 'searches', 'carts', 'invoices']
for table_name in table_names:
    dbo_obj.run_query(table_name=table_name)
print("Export Successful Completed \nDatabase Connection Closed")
dbo_obj.mysql_disconnect()

Export Successful Completed 
Database Connection Closed
